In [1]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
train = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/skillbox-computer-vision-project/sample_submission.csv')
test.head()

In [ ]:
print("Кол-во классов: {}".format(train['emotion'].nunique()))

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = 224

image_gen = ImageDataGenerator() 

train_generator = image_gen.flow_from_dataframe(train,
                                                 directory=str('/kaggle/input/train-data'),
                                                 x_col='image_path',
                                                 y_col='emotion',
                                                 batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                class_mode='categorical')


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax'),
    ])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
%%timeit -n 10 -r 10

test = model(np.random.rand(1,224,224,3).astype(np.float32))

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, epochs=10, batch_size=BATCH_SIZE)

In [ ]:
predict_dict = {0:'anger',1:'contempt',2:'disgust',3:'fear',4:'happy',5:'neutral',6:'sad',7:'surprise',8:'uncertain'}

In [5]:
test_data = tf.keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/test-data',
    batch_size=BATCH_SIZE,
    image_size=(IMAGE_SIZE,IMAGE_SIZE)
)

Found 5000 files belonging to 1 classes.


In [ ]:
emotion = model.predict(test_data)

In [ ]:
emotion_cm = [predict_dict[np.argmax(i)] for i in emotion]

In [ ]:
test['emotion'] = emotion_cm
test.head()

In [ ]:
test.to_csv('/kaggle/working/submission.csv', sep=',', index=False)

В моей первой версии ноутбука я использовал простую модель, которая была использована в одном из прошлых заданий этого курса. Здесь я просто хотел проверить получится ли успешно запустить ноутбук в kaggle. Также стоит заметить, что я неправильно создал датасет тестовых данных и как следствие точность предсказания модели была неправильной.